# Entraînement d'un modèle multilingue de classification de texte (IMDb + Allociné)

Ce notebook implémente un pipeline complet de classification binaire de sentiments à partir de textes en anglais (IMDb) et en français (Allociné). Il inclut :
- Le nettoyage des données textuelles,
- La fusion et la préparation des jeux de données,
- L'utilisation d'un tokenizer multilingue (`distilbert-base-multilingual-cased`),
- L'entraînement avec `Trainer` de HuggingFace,
- L'évaluation de la performance du modèle,
- Une démonstration de prédiction.


## Importation des bibliothèques nécessaires

On commence par importer les bibliothèques utiles pour le traitement des données, le modèle pré-entraîné et l'entraînement :


In [2]:
from datasets import load_dataset, concatenate_datasets
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments, AutoTokenizer
import numpy as np
import re
from evaluate import load
import torch
from torch.nn import functional as F

## Nettoyage du texte et prétraitement

Avant d'entraîner un modèle de classification, il est important de nettoyer les textes pour enlever les balises HTML, les caractères spéciaux et les espaces inutiles. On définit ci-dessous une fonction `clean_text` utilisée dans deux fonctions de prétraitement : une pour les textes en anglais (IMDb) et une autre pour le français (Allociné).


In [ ]:
def clean_text(text):
    text = re.sub(r'<br\s*/?>', ' ', text)
    text = re.sub(r'[^a-zA-ZÀ-ÿ0-9,.!?\'\" ]+', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    return text.lower().strip()

def preprocess_function_en(examples):
    examples["text"] = [clean_text(t) for t in examples["text"]]
    return examples

def preprocess_function_fr(examples):
    examples["text"] = [clean_text(t) for t in examples["review"]]
    return examples

## Chargement et prétraitement des jeux de données

Nous utilisons deux jeux de données pour l'entraînement du modèle :

- **IMDb** : pour les critiques en anglais
- **Allociné** : pour les critiques en français

Chaque dataset est nettoyé avec la fonction `clean_text` définie précédemment. Le jeu de données Allociné ne contient pas de séparation explicite entre entraînement et test. Nous réalisons donc une séparation manuelle à l'aide de `train_test_split`.



In [ ]:
dataset_en = load_dataset("imdb")
dataset_fr = load_dataset("allocine")

dataset_en = dataset_en.map(preprocess_function_en, batched=True)
dataset_fr = dataset_fr.map(preprocess_function_fr, batched=True)

dataset_fr = dataset_fr["train"].train_test_split(test_size=0.2, seed=42)
dataset_fr = {
    "train": dataset_fr["train"],
    "test": dataset_fr["test"]
}

## Fusion des jeux de données multilingues

Nous fusionnons les jeux de données anglais (IMDb) et français (Allociné) pour créer un ensemble multilingue. L’objectif est d’entraîner un modèle capable de gérer les deux langues simultanément.

- Les jeux d'entraînement sont concaténés et mélangés.
- Une partie des données d'entraînement est utilisée comme jeu de validation (20%).
- Les jeux de test sont également concaténés pour évaluer les performances globales.



In [ ]:
def merge_datasets(ds1, ds2):
    train = concatenate_datasets([ds1["train"], ds2["train"]]).shuffle(seed=42)
    test = concatenate_datasets([ds1["test"], ds2["test"]]).shuffle(seed=42)
    train_valid_split = train.train_test_split(test_size=0.2, seed=42)
    return train_valid_split["train"], train_valid_split["test"], test

train_dataset, valid_dataset, test_dataset = merge_datasets(dataset_en, dataset_fr)

## Tokenisation des textes

Nous utilisons un **tokenizer multilingue** (`distilbert-base-multilingual-cased`) pour convertir les textes en vecteurs numériques utilisables par le modèle.

- Le texte est tronqué et remplit jusqu'à une longueur maximale de 256 tokens.
- La tokenisation est appliquée par lot (`batched=True`) pour plus d'efficacité.


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-multilingual-cased")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=256)

train_dataset = train_dataset.map(tokenize_function, batched=True)
valid_dataset = valid_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

## Préparation des datasets pour PyTorch

Pour faciliter l’entraînement avec `transformers` et PyTorch, nous :

- Supprimons la colonne `text` qui n’est plus nécessaire après tokenisation.
- Renommons la colonne `label` en `labels` pour correspondre à l’API du modèle.
- Convertissons les datasets au format PyTorch (`set_format("torch")`).


In [ ]:
def prepare_dataset(ds):
    if "text" in ds.column_names:
        ds = ds.remove_columns(["text"])
    ds = ds.rename_column("label", "labels")
    ds.set_format("torch")
    return ds

train_dataset = prepare_dataset(train_dataset)
valid_dataset = prepare_dataset(valid_dataset)
test_dataset = prepare_dataset(test_dataset)

## Chargement du modèle et configuration de l'entraînement

- Chargement du modèle **DistilBERT multilingue** pré-entraîné, adapté pour une classification binaire (`num_labels=2`).
- Configuration des paramètres d’entraînement via `TrainingArguments` :
  - Dossier de sortie `./results`
  - Taux d’apprentissage de `2e-5`
  - Batch size de 16 pour entraînement et évaluation
  - Entraînement sur 3 epochs
  - Décroissance du poids (weight decay) pour régularisation
  - Logs sauvegardés dans `./logs`
  - Pas de rapport externe (`report_to=[]`)



In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-multilingual-cased", num_labels=2)

training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=500,
    report_to=[],
)

## Définition de la métrique d’évaluation

Nous utilisons la métrique d’**accuracy** pour évaluer les performances du modèle.  
La fonction `compute_metrics` prend en entrée les logits du modèle et les labels réels,  
puis calcule la précision des prédictions.



In [ ]:
metric = load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

## Entraînement et évaluation du modèle

- Création d’un objet `Trainer` avec le modèle, les arguments d’entraînement, les datasets d'entraînement et de validation, et la fonction d’évaluation.
- Lancement de l’entraînement avec `trainer.train()`.
- Évaluation sur le dataset de validation.
- Affichage de la précision (accuracy) obtenue.

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()
eval_result = trainer.evaluate()
print(f"Accuracy sur validation: {eval_result['eval_accuracy']:.4f}")

## Sauvegarde du modèle et du tokenizer

Après l’entraînement, on sauvegarde le modèle et le tokenizer  
dans un dossier local pour une utilisation future.

In [ ]:
model.save_pretrained("./my_model")
tokenizer.save_pretrained("./my_model")

## Test de prédiction sur un exemple

On passe le modèle en mode évaluation,  
on prépare un texte d’exemple,  
on le nettoie, on le tokenize,  
puis on obtient la prédiction avec confiance.

In [ ]:
model.eval()
review_text = "Le film était incroyable et très émouvant."
cleaned_text = clean_text(review_text)
inputs = tokenizer(cleaned_text, return_tensors="pt", truncation=True, padding=True, max_length=256)

with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits
    probs = F.softmax(logits, dim=-1)
    predicted_class = torch.argmax(probs, dim=-1).item()

labels_map = {0: "négatif", 1: "positif"}
print(f"Texte : {review_text}")
print(f"Prédiction : {labels_map[predicted_class]} (confiance : {probs[0][predicted_class]:.2f})")